<a href="https://colab.research.google.com/github/indranildchandra/Online-Patient-Conversation-Classifier/blob/master/src/Online_Patient_Conversation_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving test.csv to test.csv
Saving train.csv to train.csv
User uploaded file "test.csv" with length 1174648 bytes
User uploaded file "train.csv" with length 2378727 bytes


In [0]:
#Install required dependencies
!pip install tensorflow==1.12.0

In [0]:
# import all libraries/dependencies
from tensorflow.contrib import learn
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime
import time
import csv
import re
import os

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [0]:
def generate_dataset():
	train_data_df = pd.read_csv("train.csv", encoding="ISO-8859-1")
	test_data_df = pd.read_csv("test.csv", encoding="ISO-8859-1")

	positive_train_data_df = train_data_df[train_data_df['Patient_Tag'] == 1]
	positive_train_data = positive_train_data_df[['TRANS_CONV_TEXT']]

	negative_train_data_df = train_data_df[train_data_df['Patient_Tag'] == 0]
	negative_train_data = negative_train_data_df[['TRANS_CONV_TEXT']]

	test_data = test_data_df[['TRANS_CONV_TEXT']]

	for index, row in positive_train_data.iterrows():
		file_handler1 = open("patient_conversation-positive.txt", 'w', encoding='utf-8')
		file_handler1.write(str(row[0])+"\n")
		file_handler1.close()

	for index, row in negative_train_data.iterrows():
		file_handler2 = open("patient_conversation-negative.txt", 'w', encoding='utf-8')
		file_handler2.write(str(row[0])+"\n")
		file_handler2.close()	

	for index, row in test_data.iterrows():
		file_handler3 = open("patient_conversations-test.txt", 'w', encoding='utf-8')
		file_handler3.write(str(row[0])+"\n")
		file_handler3.close()

In [0]:
class TextCNN(object):
    #A CNN for text classification. Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/gpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")


In [0]:
def clean_str(string):
    # Tokenization/string cleaning for all datasets.
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \\( ", string)
    string = re.sub(r"\)", " \\) ", string)
    string = re.sub(r"\?", " \\? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [0]:
def load_data_and_labels(positive_data_file, negative_data_file):
    # Loads data from files, splits the data into words and generates labels. Returns split sentences and labels.

    # Load data from files
    positive_examples = list(open(positive_data_file, "r", encoding="utf8").readlines())
    positive_examples = [s.strip() for s in positive_examples]
    negative_examples = list(open(negative_data_file, "r", encoding="utf8").readlines())
    negative_examples = [s.strip() for s in negative_examples]
    # Split by words
    x_text = positive_examples + negative_examples
    x_text = [clean_str(sent) for sent in x_text]
    # Generate labels
    positive_labels = [[0, 1] for _ in positive_examples]
    negative_labels = [[1, 0] for _ in negative_examples]
    y = np.concatenate([positive_labels, negative_labels], 0)
    return [x_text, y]

In [0]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    # Generates a batch iterator for a dataset.
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]


In [0]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]  
    print("\nDeleting Following keys...")
    for keys in keys_list:
        print(keys)
        FLAGS.__delattr__(keys)

In [0]:
def print_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]  
    print("\nCreated Following keys...")
    for keys in keys_list:
        print(keys)

In [0]:
# Parameters
# ==================================================

# Delete all flags before declaring
del_all_flags(tf.flags.FLAGS)

# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("positive_data_file", "patient_conversation-positive.txt", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "patient_conversation-negative.txt", "Data source for the negative data.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
tf.flags.DEFINE_string("model_path", "", "Path where the model is saved.")

FLAGS = tf.flags.FLAGS

# Print all flags after declaring
print_all_flags(FLAGS)


Deleting Following keys...
test_data_file
checkpoint_dir
eval_train
batch_size
allow_soft_placement
log_device_placement

Created Following keys...
dev_sample_percentage
positive_data_file
negative_data_file
embedding_dim
filter_sizes
num_filters
dropout_keep_prob
l2_reg_lambda
batch_size
num_epochs
evaluate_every
checkpoint_every
num_checkpoints
allow_soft_placement
log_device_placement
model_path


In [0]:
def preprocess():
    # Data Preparation
    # ==================================================

    # Load data
    print("Loading data...")
    x_text, y = load_data_and_labels(FLAGS.positive_data_file, FLAGS.negative_data_file)

    # Build vocabulary
    max_document_length = max([len(x.split(" ")) for x in x_text])
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
    x = np.array(list(vocab_processor.fit_transform(x_text)))

    # Randomly shuffle data
    np.random.seed(10)
    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_shuffled = x[shuffle_indices]
    y_shuffled = y[shuffle_indices]

    # Split train/test set
    # TODO: This is very crude, should use cross-validation
    dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))
    x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
    y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

    del x, y, x_shuffled, y_shuffled

    print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
    print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
    return x_train, y_train, vocab_processor, x_dev, y_dev

In [0]:
def restore(sess_var, model_path):
    if model_path is not None:
        if os.path.exists("{}.index".format(model_path)):
            saver = tf.train.Saver(var_list=tf.trainable_variables())
            saver.restore(sess_var, model_path)
            print("Model at %s restored" % model_path)
        else:
            print("Model path does not exist, skipping...")
    else:
        print("Model path is None - Nothing to restore")

In [0]:
def train(x_train, y_train, vocab_processor, x_dev, y_dev):
    # Training
    # ==================================================

    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
          allow_soft_placement=FLAGS.allow_soft_placement,
          log_device_placement=FLAGS.log_device_placement)
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = TextCNN(
                sequence_length=x_train.shape[1],
                num_classes=y_train.shape[1],
                vocab_size=len(vocab_processor.vocabulary_),
                embedding_size=FLAGS.embedding_dim,
                filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
                num_filters=FLAGS.num_filters,
                l2_reg_lambda=FLAGS.l2_reg_lambda)

            # Define Training procedure
            global_step = tf.Variable(0, name="global_step", trainable=False)
            optimizer = tf.train.AdamOptimizer(1e-3)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Keep track of gradient values and sparsity (optional)
            grad_summaries = []
            for g, v in grads_and_vars:
                if g is not None:
                    grad_hist_summary = tf.summary.histogram("{}/grad/hist".format((v.name).replace(":","_")), g)
                    sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format((v.name).replace(":","_")), tf.nn.zero_fraction(g))
                    grad_summaries.append(grad_hist_summary)
                    grad_summaries.append(sparsity_summary)
            grad_summaries_merged = tf.summary.merge(grad_summaries)

            # Output directory for models and summaries
            timestamp = str(int(time.time()))
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
            print("Writing to {}\n".format(out_dir))

            # Summaries for loss and accuracy
            loss_summary = tf.summary.scalar("loss", cnn.loss)
            acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

            # Train Summaries
            train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
            train_summary_dir = os.path.join(out_dir, "summaries", "train")
            train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

            # Dev summaries
            dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
            dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
            dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

            # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
            checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
            checkpoint_prefix = os.path.join(checkpoint_dir, "model")
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

            # Write vocabulary
            vocab_processor.save(os.path.join(out_dir, "vocab"))

            # Initialize all variables
            sess.run(tf.global_variables_initializer())

            def train_step(x_batch, y_batch):
                """
                A single training step
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
                }
                _, step, summaries, loss, accuracy = sess.run(
                    [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                train_summary_writer.add_summary(summaries, step)

            def dev_step(x_batch, y_batch, writer=None):
                """
                Evaluates model on a dev set
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: 1.0
                }
                step, summaries, loss, accuracy = sess.run(
                    [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                if writer:
                    writer.add_summary(summaries, step)

            # Generate batches
            batches = batch_iter(
                list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
            # Training loop. For each batch...
            for batch in batches:
                x_batch, y_batch = zip(*batch)
                train_step(x_batch, y_batch)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % FLAGS.evaluate_every == 0:
                    print("\nEvaluation:")
                    dev_step(x_dev, y_dev, writer=dev_summary_writer)
                    print("")
                if current_step % FLAGS.checkpoint_every == 0:
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                    print("Saved model checkpoint to {}\n".format(path))
                    FLAGS.model_path = path
                    
        #restore(sess, FLAGS.model_path)

In [0]:
def test(model_path):
    # Delete all flags before declaring
    del_all_flags(tf.flags.FLAGS)
    
    tf.flags.DEFINE_string("test_data_file", "patient_conversations-test.txt", "Data source for the test data.")
    tf.flags.DEFINE_string("checkpoint_dir", model_path, "Checkpoint directory from training run")
    tf.flags.DEFINE_boolean("eval_train", True, "Evaluate on all training data")
    tf.flags.DEFINE_integer("batch_size", 2, "Batch Size (default: 64)")
    tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
    tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
    FLAGS = tf.flags.FLAGS
    
    # Print all flags after declaring
    print_all_flags(FLAGS)
    
    if FLAGS.eval_train:
        # Load data from files
        test_examples = list(open(FLAGS.test_data_file, "r", encoding="utf8").readlines())
        test_examples = [s.strip() for s in test_examples]
        # Split by words
        x_raw = [clean_str(sent) for sent in test_examples]
    else:
        x_raw = ["I think I am suffering from cold and flu", "I am really loving this problem"]
        
    
    
    # Map data into vocabulary
    vocab_path = os.path.join(FLAGS.checkpoint_dir, "./../../", "vocab")
    vocab_processor = learn.preprocessing.VocabularyProcessor.restore(vocab_path)
    x_test = np.array(list(vocab_processor.transform(x_raw)))
    print("\nEvaluating...\n")
    
    # Evaluation
    # ==================================================
    checkpoint_file = tf.train.latest_checkpoint(os.path.join(FLAGS.checkpoint_dir, "./../"))
    graph = tf.Graph()
    with graph.as_default():
        session_conf = tf.ConfigProto(allow_soft_placement=FLAGS.allow_soft_placement, log_device_placement=FLAGS.log_device_placement)
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            # Load the saved meta graph and restore variables
            saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
            saver.restore(sess, checkpoint_file)

            # Get the placeholders from the graph by name
            input_x = graph.get_operation_by_name("input_x").outputs[0]
            # input_y = graph.get_operation_by_name("input_y").outputs[0]
            dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

            # Tensors we want to evaluate
            predictions = graph.get_operation_by_name("output/predictions").outputs[0]

            # Generate batches for one epoch
            batches = batch_iter(list(x_test), FLAGS.batch_size, 1, shuffle=False)

            # Collect the predictions here
            patient_tag = ["Patient_Tag"]

            for x_test_batch in batches:
                batch_predictions = sess.run(predictions, {input_x: x_test_batch, dropout_keep_prob: 1.0})
                patient_tag = np.concatenate([patient_tag, batch_predictions])
       
    patient_index = ["Index"]
    patient_index = np.concatenate([patient_index, np.arange(1,len(x_raw)+1,1)])

    conversation_data = ["Conversation_Data"]
    conversation_data = np.concatenate([conversation_data, np.array(x_raw)])
    
    # Save the evaluation to a csv
    predictions = np.column_stack((patient_index, patient_tag))
    predictions_description = np.column_stack((conversation_data, patient_tag))

    submission_file = os.path.join(FLAGS.checkpoint_dir, "../../../../", "prediction.csv")
    predictions_description_file = os.path.join(FLAGS.checkpoint_dir, "../../../../", "prediction-description.csv")

    out_path = os.path.abspath(submission_file)
    print("Saving evaluation to {0}".format(out_path))
    with open(out_path, 'w+', encoding="utf8", newline='') as f:
        csv.writer(f).writerows(predictions)

    out_path = os.path.abspath(predictions_description_file)
    print("Saving prediction descriptions to {0}".format(out_path))
    with open(out_path, 'w+', encoding="utf8", newline='') as f:
        csv.writer(f).writerows(predictions_description)

In [0]:
def main(argv=None):
    generate_dataset()
    x_train, y_train, vocab_processor, x_dev, y_dev = preprocess()
    train(x_train, y_train, vocab_processor, x_dev, y_dev)
    test(FLAGS.model_path)

In [0]:
if __name__ == '__main__':
    tf.app.run()
    os._exit(1)

Loading data...
Vocabulary Size: 24377
Train/Dev split: 2083/231
Writing to /content/runs/1548136699

2019-01-22T05:58:21.014785: step 1, loss 1.88718, acc 0.671875
2019-01-22T05:58:21.715057: step 2, loss 2.25637, acc 0.734375
2019-01-22T05:58:22.406838: step 3, loss 0.914202, acc 0.84375
2019-01-22T05:58:23.093760: step 4, loss 3.15211, acc 0.671875
2019-01-22T05:58:23.783852: step 5, loss 1.45498, acc 0.75
2019-01-22T05:58:24.471676: step 6, loss 2.02851, acc 0.765625
2019-01-22T05:58:25.159749: step 7, loss 2.03343, acc 0.71875
2019-01-22T05:58:25.847648: step 8, loss 1.55283, acc 0.703125
2019-01-22T05:58:26.537528: step 9, loss 1.26555, acc 0.765625
2019-01-22T05:58:27.227826: step 10, loss 1.68106, acc 0.640625
2019-01-22T05:58:27.915624: step 11, loss 1.09731, acc 0.78125
2019-01-22T05:58:28.605687: step 12, loss 1.95123, acc 0.671875
2019-01-22T05:58:29.297329: step 13, loss 1.80324, acc 0.578125
2019-01-22T05:58:29.982330: step 14, loss 1.88467, acc 0.71875
2019-01-22T05:58:3

SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [0]:
# Download prediction.csv file
files.download("prediction.csv")
files.download("prediction-description.csv")